 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8967/notebooks/scrape_old_weather_data.ipynb#scrape-APEX-historical-page" data-toc-modified-id="scrape-APEX-historical-page-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>scrape APEX historical page</a></span></li><li><span><a href="http://localhost:8967/notebooks/scrape_old_weather_data.ipynb#Analysis" data-toc-modified-id="Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analysis</a></span><ul class="toc-item"><li><span><a href="http://localhost:8967/notebooks/scrape_old_weather_data.ipynb#wind-speeds-over-time" data-toc-modified-id="wind-speeds-over-time-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>wind speeds over time</a></span></li><li><span><a href="http://localhost:8967/notebooks/scrape_old_weather_data.ipynb#look-at-month/year-trends" data-toc-modified-id="look-at-month/year-trends-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>look at month/year trends</a></span></li></ul></li></ul></div>

In [2]:
import matplotlib
%matplotlib inline 

import matplotlib.pyplot as plt
import datetime
import requests
import urllib
import time
from bs4 import BeautifulSoup
import os
import pandas as pd
import zipfile
import seaborn as sns
import os


url="http://www.submm.caltech.edu/submm.org/site/weather/"


# scrape APEX historical page

In [3]:
url="http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=23-JUL-2006&Submit=Send"
response = requests.get(url=url)
soup = BeautifulSoup(response.text, "html.parser")
print "pulling data from {0}".format(url)
#datalinks = [link["href"] for link in soup.findAll('a') if "txt" in link.attrs["href"]]

pulling data from http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=23-JUL-2006&Submit=Send


In [4]:
var_lookup = {}
var_lookup["t1"] = "Humidity"
var_lookup["t2"] = "Temperature"
var_lookup["t3"] = "DewPoint"
#var_lookup["t4"] = "Humidity"
var_lookup["t5"] = "RadiometerData"
var_lookup["goog2"] = "WindSpeed"
#var_lookup["s1"] = "AverageWindSpeed"

var_values = {}


In [5]:
import re
def parse_weather_data_from_apex_webpage(soup):
    data = soup.find_all("script")
    var_tables = []
    table_all = None    
    for text in data:
        if text.string is None:
            continue
        for line in text.string.split("\n"):
            if "var" in str(line):
                p = re.compile('var (.*?) = (\[\[.*?);')
                m = p.match(line)
                if m is not None:
                    var, values = m.groups()
                    if var in var_lookup:
                        var_name = var_lookup[var]
                        var_values[var_name] = values.replace("[[","").replace("]]","").replace("\\","").split("],[")
                        table = pd.DataFrame(var_values[var_name])[0].str.split(",", expand=True).rename(columns={0:'time', 1:var_name})
                        table["time"] = pd.to_datetime(table["time"],format="\"%m/%d/%Y %H:%M\"")
                        table[var_name] = pd.to_numeric(table[var_name],errors='coerce')
                        table.set_index("time",inplace=True)
                        if table_all is None:
                            table_all = table
                        else:
                            table_all  = table_all.join(table[var_name])
                        var_tables.append(table)
            #
    if table_all is None:
        return
    else:
        return table_all.groupby(["time"]).mean() 

#
def send_table_to_database(table,table_name="APEX_weather_data"):
    #host="http://localhost"
    #port=8068
    host="https://logging.batleth.ph1.uni-koeln.de"
    port=""
    #
    user = ''
    password = ''
    dbname = 'CCAT'
    send_every_nth_row = 100
    #
    if port == "":
        url = "{0}/write?db={1}".format(host,dbname)
    else:
        url = "{0}:{1}/write?db={2}".format(host,port,dbname)
    #
    time_column="time"
    
    tags=[]
    skip_columns = ["date","time",None,time_column]
    value_columns = list(set(table.columns)-set(skip_columns)-set(tags))
    influx_messages=[]
    #
    for i,(ts,row) in enumerate(table.iterrows()):
        tags = ",".join(["{0}={1}".format(tag,row.get(tag)) for tag in tags])
        values = ",".join(["{0}={1}".format(tag,row.get(tag)) for tag in value_columns])
        if tags=="":
            influx_message="{0} {1}{2:20.0f}".format(table_name, values, float(ts.strftime("%s"))*1e9)
        else:
            influx_message="{0},{1} {2}{3:18.0f}".format(table_name, tags, values, float(ts.strftime("%s"))*1e9)
        #
        influx_messages.append(influx_message)
        #
        if i%send_every_nth_row==0:
            #print "{0}/{1}".format(i,len(table))
            #print "\n".join(influx_messages)
            r = requests.post(url, data="\n".join(influx_messages))
            influx_messages=[]
#table = parse_weather_data_from_apex_webpage(soup)

NameError: name 'date' is not defined

In [8]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2013, 7, 4)
end_date = date(2019, 8, 1)
#start_date = date(2006, 11, 22)
#end_date = date(2006, 11, 23)
tables = []
for days, single_date in enumerate(daterange(start_date, end_date)):
    url="http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate={0}&Submit=Send".format(single_date.strftime("%d-%b-%Y"))
    print "{0}/{1} {2}".format(days,(end_date-start_date).days,url)
    response = requests.get(url=url)
    soup = BeautifulSoup(response.text, "html.parser")
    table = parse_weather_data_from_apex_webpage(soup)
    if table is not None:
        send_table_to_database(table,table_name="APEX_weather_data")
    else:
        print "no data found"
    time.sleep(2)
    #tables.append()
#table_all = pd.concat(tables)
#table_all.drop_duplicates(inplace=True)

0/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=04-Jul-2013&Submit=Send
1/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=05-Jul-2013&Submit=Send
2/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=06-Jul-2013&Submit=Send
3/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=07-Jul-2013&Submit=Send
4/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=08-Jul-2013&Submit=Send
5/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=09-Jul-2013&Submit=Send
6/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=10-Jul-2013&Submit=Send
7/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=11-Jul-2013&Submit=Send
8/2219 http://www.apex-telescope.org/weather/Historical_weather/index.htm?utdate=12-Jul-2013&Submit=Send
9/2219 http://www.apex-telescope.org/weather/Historical

In [ ]:
table_all

In [ ]:
table.drop_duplicates(inplace=True)
fig,ax = plt.subplots(1,1)
fig.set_size_inches(15,15)
ax.plot_date(table.index, table["WindSpeed"])
ax.plot_date(table.index, table.RadiometerData)

In [ ]:
#
for var_name in ["Humidity"]:
    table = pd.DataFrame(var_values[var_name])[0].str.split(",", expand=True).rename(columns={0:'time', 1:var_name})
    table["time"] = pd.to_datetime(table["time"],format="\"%m/%d/%Y %H:%M\"")
    table[var_name] = pd.to_numeric(table[var_name])


In [ ]:
dt_str = table.time.values[4]
print dt_str
datetime.datetime.strptime(dt_str,"\"%m/%d/%Y %H:%M\"")

In [ ]:
names=["unit","date","time","wind_speed", "wind_direction", 
       "wind_variation","wind_speed_peak", "temperature",
       "relative_humidity","pressure","battery","zero",None]
tables = []
for i,filename in enumerate(datalinks):
    print "extracting ({0}/{1}) {2}".format(i,len(datalinks),filename)
    #
    table = pd.read_csv(filename.replace(".zip",""),names=names)
    for index,row in table.iterrows():
        try:
            row.date = datetime.datetime.strptime(x,'%y/%m/%d').strftime('%y/%m/%d')
        except:
            row.date = "nan"
    #table['date'] = table['date'].apply(lambda x: datetime.datetime.strptime(x,'%y/%m/%d').strftime('%y/%m/%d')if "/" in x  else float("nan"))
    table.dropna(subset=["date","time"],inplace=True)
    tables.append(table)

table = pd.concat(tables)
del(tables)
table["ts"] = pd.to_datetime(table['date'] + ' ' + table['time'], format='%y/%m/%d %H:%M:%S', errors='ignore')
table.drop_duplicates(inplace=True)
table["timestamp"] = table["ts"].copy()
table.set_index("ts",inplace=True, append=True)
#
for column in ["temperature","wind_speed_peak","pressure","relative_humidity"]:
    table[column] = pd.to_numeric(table[column],errors='coerce')
    table.dropna(subset=[column],axis=0,inplace=True)
    

In [ ]:
table_filename = "weather_data_2006_to_2014.csv"
#
print "saving combimed data to {0}".format(table_filename)
table.to_csv(table_filename)

# Analysis
## wind speeds over time

In [ ]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(15,5)
ax.plot_date(table.timestamp,table.wind_speed_peak,ms=0.1)
ax.set_ylim(0,40)



## look at month/year trends

In [ ]:

table.drop_duplicates(inplace=True)
#table.dropna(subset=["temperature"],inplace=True)
#table.dropna(subset=["wind_speed"],inplace=True)
#pivot_table = table.pivot("month", "year", "wind_speed_peak")

for column in ["temperature","wind_speed_peak","pressure","relative_humidity"]:
    fig, ax = plt.subplots(1,1)
    fig.suptitle(column)    
    fig.set_size_inches(15,7)
    pivot_table = table.groupby([table.timestamp.dt.month, table.timestamp.dt.year])[column].median().unstack()
    sns.heatmap(pivot_table)
    ax.set_xlabel("Year")
    ax.set_ylabel("Month")

#ax = sns.heatmap(pivot_table)

In [ ]:
pivot_table = table.groupby([table.timestamp.dt.month, table.timestamp.dt.year])["wind_speed"].median().unstack()
ax = sns.heatmap(pivot_table)

In [ ]:
pivot_table

In [ ]:
# write to cologne db
test

host="https://logging.batleth.ph1.uni-koeln.de"
port=""
# write to a local influxdb
host="http://localhost"
port=8068
#
user = ''
password = ''
dbname = 'CCAT'
send_every_nth_row = 100
#
if port == "":
    url = "{0}/write?db={1}".format(host,dbname)
else:
    url = "{0}:{1}/write?db={2}".format(host,port,dbname)
#
time_column="timestamp"
table_name="CCAT_weather_data"
tags=[]
skip_columns = ["date","time",None,time_column]
value_columns = list(set(table.columns)-set(skip_columns)-set(tags))
influx_messages=[]
#
for i,((index,ts),row) in enumerate(table.iterrows()):
    print i
    tags = ",".join(["{0}={1}".format(tag,row.get(tag)) for tag in tags])
    values = ",".join(["{0}={1}".format(tag,row.get(tag)) for tag in value_columns])
    if tags=="":
        influx_message="{0} {1}{2:20.0f}".format(table_name, values, float(row.timestamp.strftime("%s"))*1e9)
    else:
        influx_message="{0},{1} {2}{3:18.0f}".format(table_name, tags, values, float(row.timestamp.strftime("%s"))*1e9)
    #
    influx_messages.append(influx_message)
    #
    if i%send_every_nth_row==0:
        print "{0}/{1}".format(index,len(table))
        #print "\n".join(influx_messages)
        r = requests.post(url, data="\n".join(influx_messages))
        influx_messages=[]